In [ ]:
#from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from sentence_transformers import SentenceTransformer
#from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
#from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

: 

In [2]:
from bs4 import BeautifulSoup
import requests
from langchain_community.document_loaders import WebBaseLoader

# ✅ Step 1: Get all links from the webpage
url = "https://brainlox.com/courses/category/technical"
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")

# ✅ Step 2: Extract all links from <a> tags
links = []
for a_tag in soup.find_all("a", href=True):
    link = a_tag["href"]
    if "brainlox.com" in link:  # Ensure it's an internal link
        links.append(link)

print(f"Found {len(links)} links.")

# ✅ Step 3: Load data from each link using WebBaseLoader
all_docs = []
for link in links[:5]:  # Limiting to 5 links for testing
    try:
        print(f"Loading: {link}")
        loader = WebBaseLoader(link)
        docs = loader.load()
        all_docs.extend(docs)
    except Exception as e:
        print(f"Failed to load {link}: {e}")

# ✅ Step 4: Print extracted content (first 500 chars from first document)
if all_docs:
    print("\nExtracted Content:\n", all_docs[0].page_content[:500])


Found 1 links.
Loading: mailto:support@brainlox.com
Failed to load mailto:support@brainlox.com: No connection adapters were found for 'mailto:support@brainlox.com'


In [3]:


# Use a markdown file from github page
loader = WebBaseLoader("https://brainlox.com/courses/category/technical")

docs = loader.load()
print(docs[0].page_content[:500])

Brainlox: Learn technical courses.Courses TechnicalAcademicLanguageMusicLifestyleBook a Free Demo NowSign InFAQContact UsHomeCoursesCoursesWe found great courses available for you$30per sessionLEARN SCRATCH PROGRAMING
Scratch Course is the foundation of coding and is a building block of a coding journey. If you want 16 LessonsView Details$30per sessionLEARN CLOUD COMPUTING BASICS-AWS
In this course we are going to cover the basics and the most important services on AWS,
At the end  20 LessonsVie


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

#Create a split of the document using the text splitter
splits = text_splitter.split_documents(docs)
# Extract text from Document objects
splits_text = [doc.page_content for doc in splits]


In [7]:
model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

OSError: The paging file is too small for this operation to complete. (os error 1455)

In [8]:
persist_directory = "docs/chroma/"

# ✅ Ensure you have an embedding model
#embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# ✅ Convert Documents to Text
#texts = [doc.page_content for doc in splits]  # Extract text from Document objects

# ✅ Store in ChromaDB
vectordb = Chroma.from_texts(
    texts=splits_text,
    embedding=model,
    persist_directory=persist_directory
)

# ✅ Print the number of stored vectors
print("Number of stored embeddings:", vectordb._collection.count())

NameError: name 'model' is not defined

In [ ]:
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

# ✅ Load a local model using Transformers
#model_pipeline = pipeline("text-generation", model="facebook/opt-1.3b",max_new_tokens=200,)
model_pipeline = pipeline("text-generation", model="facebook/opt-350m", max_new_tokens=200)


# ✅ Define llm using HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=model_pipeline)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 2})
)



Device set to use cpu


In [ ]:
# Pass question to the qa_chain
question = "What are major courses in python"
result = qa_chain.invoke({"query": question})
#result["result"]
print(result["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

This introduction to cloud computing on Amazon AWS course takes you from the AWS Ad 18 LessonsView Details$30per sessionPYTHON PROGRAMMING-BEGINNER
Python is a language with simple syntax, and a powerful set of libraries. It has a rich programming 16 LessonsView Details$30per sessionRoblox Programming For BeginnersExplore the dynamic universe of game development with our "Roblox Game Development Fundamentals" cou 15 LessonsView Details$32per sessionPYTHON PROGRAMMING-INTERMEDIATE
Take your python skills to the next level and start building real applications.
Python is a pro 16 LessonsView Details$35per sessionPYTHON PROGRAMMING-ADVANCEIf you already know Python basics, then this training is the next step in your Python learning path  30 LessonsView Details$30per sessionPYTHON PROGRAMMING GROUP CLASSES - BEGINNER

This introd